# Get all counties that within a 50 mile radius from "x" county

In [ ]:
def validcounty(x,mileage=50):
    import pandas as pd, geopy
    from geopy.distance import vincenty
    
    theyes = []   
    county_miles = dict()
    mileslimit = mileage + 1
    region_dic = dict()   
    resultslist = [] 
    
        # maindata represents the state requesting the analysis
    for i in maindata:
        # grab each county in maindata list and compared it to "x" county from neighboring state
        # apply vincenty algorithm and convert answer to miles
        result = vincenty(x, maindata[i]).miles
        
        # If true, then capture counties that meet the mileslimit requirement
        if result <= mileslimit:
            theanswer = True
            county = i           
        #conver result to integer and then into a string
            miles = str(int((result))) + ' mi'           
            resultado = miles + ' away from ' + county
        # appends the number of counties that fit the mileslimit radious into theyes list
            theyes.append(resultado)
        # dict created (county_miles) showing county within the mileslimit radius + miles ex. 'Marathon County': '33 mi' 
            if county in county_miles:
                continue
            else:               
                county_miles[county] = miles
       

    if len(theyes)>0:        
    #using all the counties inside the radius, combine both dicts county_miles and regions to form "county:region"
        for county in county_miles:    
            for thecounty, theregion in regions.items():
                if thecounty.lower() == county.lower():
    # build new dictionary combining the county, region values. This will show all external counties that 
    # meet the mileslimit radius requirement. 
                    region_dic[thecounty] = theregion 
   
    # if the key values in region_dic and county_miles match, then save "county_miles's value + mi away from + region_couny +
    # region " into resulslist. ex. 33 mi away from Clark County, Central Region.
        for region_county, region_value in region_dic.items():
            for county, miles in county_miles.items():
                if county == region_county:
                    answerifyes = miles + ' away from ' + region_county +', '+ '<'+ region_value + '>'+ ' region.'
                    resultslist.append(answerifyes)
    # sort resultslist to grab the lower mile value  
        getfirstone = sorted(resultslist)
    # store the total number of items in resultslist
        total = ' [' +  str(len(theyes)) +']'
    # grab the first list item with the lower mile value.
        conclusion = getfirstone[0] + total
 
    else:
    #if coordinate not found inside radius then display 'False'
        conclusion = 'False: coordinates are outside the ' + str(mileslimit - 1) + '-mile radius.'
    return(conclusion)

 ## IMPORT DATA AND APPLY FUNCTION
 
import pandas as pd, geopy
from geopy.distance import vincenty
# Y = Wisconsin

path = 'C:/folderA/sc_distance/national_counties.csv'
df = pd.read_csv(path, encoding='latin-1', low_memory=False)

# #rename columns 
# df.rename(columns={'INTPTLAT':'latitude', 'INTPTLONG':'longitude'}, inplace=True)

df.columns = map(str.strip, df.columns)

#rename columns 
df.rename(columns={'INTPTLAT':'latitude', 'INTPTLONG':'longitude'}, inplace=True)

# retrieve and concatenate county coordinates Latitude: INTPTLAT , longitude:INTPTLONG 
coordinates = '(' + df['latitude'].astype(str) + ', ' + df['longitude'].astype(str) + ')'

# convert all column titles to lowercase
df.columns = map(str.lower, df.columns)


df.loc[df["name"] == "St. Croix County",["name"]] = "Saint Croix County" 

df.insert(1,'coordinates',coordinates)
df.insert(2, 'miles','')
df.insert(3, 'region','')
#wisconsin data
# mainstate = df.loc[(df['usps']=='WI') & (df['valid'] == 'Y'), ['name','coordinates']]

#southcarolina data
mainstate = df.loc[(df['usps'] =='SC')  & (df['valid'] == 'Y'),['name','coordinates']]

reg = df.loc[(df['valid']=='Y'), ['region2','name']]
regions = dict(zip(reg['name'],reg['region2']))

maindata = dict(zip(mainstate['name'],mainstate['coordinates']))


#apply Vincenty algorithm to coordinates column and paste result into miles column
df['miles'] = df['coordinates'].map(validcounty)

# extract region value and add it to new column
df['region'] = df['miles'].str.extract(r'<(.*?)>',expand=False)

# save result into a csv file
df.to_csv("C:/folderA/sc_distance/wi_answer.csv",index=False)
print('done')